In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('data/train_to_colab.csv')


le = LabelEncoder()
train['rendimiento_global'] = le.fit_transform(train['rendimiento_global'])

In [4]:
df = train.copy()

## Extraer año y semestre del periodo
df['periodo_academico'] = df['periodo_academico'].astype(int)

df['anio'] = df['periodo_academico'] // 10
df['semestre'] = df['periodo_academico'] % 10

## Features agregados por programa académico
program_group = df.groupby('e_prgm_academico')

df['prog_mean_indicador_1'] = df['e_prgm_academico'].map(program_group['indicador_1'].mean())
df['prog_mean_indicador_2'] = df['e_prgm_academico'].map(program_group['indicador_2'].mean())
df['prog_mean_indicador_3'] = df['e_prgm_academico'].map(program_group['indicador_3'].mean())
df['prog_mean_indicador_4'] = df['e_prgm_academico'].map(program_group['indicador_4'].mean())

df['prog_mean_rend'] = df['e_prgm_academico'].map(program_group['rendimiento_global'].mean())

## Features agregados por departamento
dept_group = df.groupby('e_prgm_departamento')

df['dept_mean_indicador'] = df['e_prgm_departamento'].map(
    dept_group[['indicador_1','indicador_2','indicador_3','indicador_4']].mean().mean(axis=1)
)

df['dept_mean_rend'] = df['e_prgm_departamento'].map(
    dept_group['rendimiento_global'].mean()
)

## Indicador promedio y dispersión
df['indicador_promedio'] = df[['indicador_1','indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_std'] = df[['indicador_1','indicador_2','indicador_3','indicador_4']].std(axis=1)

## Ratios relevantes
# Relaciones clave según correlaciones
df['ratio_1_2'] = df['indicador_1'] / (df['indicador_2'] + 1)
df['ratio_1_3'] = df['indicador_1'] / (df['indicador_3'] + 1)
df['ratio_1_4'] = df['indicador_1'] / (df['indicador_4'] + 1)

# Índice consolidado de indicadores correlacionados
df['indicador_234_mean'] = df[['indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_234_std'] = df[['indicador_2','indicador_3','indicador_4']].std(axis=1)

# Ratio entre indicador independiente e índice
df['ratio_1_234'] = df['indicador_1'] / (df['indicador_234_mean'] + 1)

# Diferencias (importante porque 2–3–4 se mueven juntos)
df['diff_2_3'] = df['indicador_2'] - df['indicador_3']
df['diff_2_4'] = df['indicador_2'] - df['indicador_4']
df['diff_3_4'] = df['indicador_3'] - df['indicador_4']



## Interacciones importantes socioeconómicas × desempeño
df['promedio_x_estrato'] = df['indicador_promedio'] * df['f_estratovivienda']
df['promedio_x_internet'] = df['indicador_promedio'] * df['f_tieneinternet']
df['promedio_x_computador'] = df['indicador_promedio'] * df['f_tienecomputador']

df['horas_x_indicadores'] = df['e_horassemanatrabaja'] * df['indicador_promedio']
df['matricula_x_indicadores'] = df['e_valormatriculauniversidad'] * df['indicador_promedio']


In [5]:
# ===================================================
# 1. DEFINIR CAT_FEATURES MANUALMENTE
# ===================================================
cat_features = [
    'e_prgm_academico', 
    'e_prgm_departamento', 
    'e_valormatriculauniversidad', 
    'e_horassemanatrabaja', 
    'f_estratovivienda', 
    'f_tieneinternet', 
    'f_educacionpadre', 
    'f_tienelavadora', 
    'f_tieneautomovil', 
    'e_pagomatriculapropio',
    'f_tienecomputador', 
    'f_educacionmadre'
]

# Convertirlas a string (CatBoost requiere esto)
for col in cat_features:
    df.loc[:, col] = df[col].astype(str)

# ===================================================
# 2. DEFINIR TARGET Y FEATURES
# ===================================================
target = 'rendimiento_global'
features = [col for col in df.columns if col != target]

X = df[features].copy()
y = df[target].copy()

# ===================================================
# 3. TRAIN / VALID SPLIT
# ===================================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Crear Pools para GPU
train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

# ===================================================
# 4. MODELO CATBOOST
# ===================================================
model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.1,
    depth=8,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    task_type='GPU',
    l2_leaf_reg=5,
    random_strength=1,
    bagging_temperature=0.5,
    boosting_type='Plain',
    od_type='Iter',
    od_wait=100,
    random_seed=42,
    verbose=200
)

# Entrenar
model.fit(train_pool, eval_set=valid_pool)

# ===================================================
# 5. MÉTRICAS
# ===================================================
preds = model.predict(X_valid)
acc = accuracy_score(y_valid, preds)

print("\n========================================")
print("ACCURACY VALID:", acc)
print("========================================\n")

# ===================================================
# 6. IMPORTANCIA DE FEATURES
# ===================================================
importances = pd.DataFrame({
    'feature': features,
    'importance': model.get_feature_importance()
}).sort_values(by='importance', ascending=False)

print(importances.head(25))


C:\Users\lina_\AppData\Local\Temp\ipykernel_4560\687975048.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6.0' '4.0' '4.0' ... '3.0' '4.0' '6.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype(str)
C:\Users\lina_\AppData\Local\Temp\ipykernel_4560\687975048.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1.0' '0.0' '4.0' ... '1.0' '1.0' '4.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype(str)
C:\Users\lina_\AppData\Local\Temp\ipykernel_4560\687975048.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['3.0' '3.0' '3.0' ... '3.0' '1.0' '4.0']' has dtype incompatible with float64, please explicitly cast to 

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version